In [51]:
import requests
import json

def generate_text(api_url, model, prompt, max_tokens=500):

    data = {
        "text_input": prompt,
        "max_tokens": max_tokens,
        "bad_words": "",
        "stop_words": "",
    }
    
    response = requests.post(f"{api_url}/v2/models/{model}/generate", 
                             json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.text}

In [1]:
# Example usage
api_url = "http://localhost:14524"  # Change this to your OpenAI-compatible server URL
model = "tensorrt_llm_bls"

In [2]:
with open("data.txt") as f:
    prompts = f.readlines()

# KV Caching

In [68]:
import time

start = time.time()
response = generate_text(api_url, model, prompts[0])
end = time.time()
print(f'{end - start:.2f} sec')

start = time.time()
response = generate_text(api_url, model, prompts[0])
end = time.time()
print(f'{end - start:.2f} sec')

9.45 sec
8.86 sec


# Inflight Batching

### No inflight batching

In [ ]:
import time

total_time = 0
for i, prompt in enumerate(prompts):
   start = time.time()
   response = generate_text(api_url, model, prompt)
   end = time.time()
   total_time += end - start
   print(f'Iteration {i}' , f'{end - start:.2f} sec')

print()
print(f"Total time: {total_time:.2f} sec")

Iteration 0 8.86 sec 
Prompt: Imagine a futuristic city where gravity works in reverse. Instead of buildings rising from the ground, they extend downward into the sky, suspended by powerful energy tethers. Streets are not paved on land but instead consist of floating platforms that drift between these hanging structures, adjusting their positions dynamically to control traffic flow. People move through the city using personal gravity harnesses, allowing them to ‘fall’ upward and downward at will, navigating between their homes, workplaces, and entertainment hubs. Public transportation consists of levitating shuttles that glide through the air, docking at various suspended stations.\nIn this world, water must be carefully controlled to avoid floating away, and food production has adapted to vertical farms anchored to the city's central gravity core. The sky, once a distant ceiling, is now the deep abyss below, making nighttime views an endless expanse of darkness speckled with the twink

### With Inflight batching

In [3]:
import asyncio
import aiohttp
import time

async def agenerate_text(api_url, model, prompt, max_tokens=500, speculative_sampling=False, num_draft_tokens=5):
    data = {
        "text_input": prompt,
        "max_tokens": max_tokens,
        "bad_words": "",
        "stop_words": "",
    }

    if speculative_sampling:
        data["num_draft_tokens"] = num_draft_tokens
        data["use_draft_logits"] = False
        data["stream"] = False

    async with aiohttp.ClientSession() as session:
        async with session.post(f"{api_url}/v2/models/{model}/generate", json=data) as response:
            if response.status == 200:
                return await response.json()
            else:
                return {"error": await response.text()}

In [73]:
start = time.time()
tasks = [
    agenerate_text(api_url, model, prompt)
    for prompt in prompts
]

responses = await asyncio.gather(*tasks)
for i, r in enumerate(responses):
    print(f"Prompt {i+1} response:", r)
print()
print(f"Total time: {time.time()-start:.2f} sec")

Prompt 1 response: {'model_name': 'tensorrt_llm_bls', 'model_version': '1', 'text_output': "Imagine a futuristic city where gravity works in reverse. Instead of buildings rising from the ground, they extend downward into the sky, suspended by powerful energy tethers. Streets are not paved on land but instead consist of floating platforms that drift between these hanging structures, adjusting their positions dynamically to control traffic flow. People move through the city using personal gravity harnesses, allowing them to ‘fall’ upward and downward at will, navigating between their homes, workplaces, and entertainment hubs. Public transportation consists of levitating shuttles that glide through the air, docking at various suspended stations.\\nIn this world, water must be carefully controlled to avoid floating away, and food production has adapted to vertical farms anchored to the city's central gravity core. The sky, once a distant ceiling, is now the deep abyss below, making nightti

# Chunked context

In [72]:
start = time.time()
tasks = [
    agenerate_text(api_url, model, prompt)
    for prompt in prompts
]

responses = await asyncio.gather(*tasks)
for i, r in enumerate(responses):
    print(f"Prompt {i+1} response:", r)
print()
print(f"Total time: {time.time()-start:.2f} sec")

Prompt 1 response: {'model_name': 'tensorrt_llm_bls', 'model_version': '1', 'text_output': "Imagine a futuristic city where gravity works in reverse. Instead of buildings rising from the ground, they extend downward into the sky, suspended by powerful energy tethers. Streets are not paved on land but instead consist of floating platforms that drift between these hanging structures, adjusting their positions dynamically to control traffic flow. People move through the city using personal gravity harnesses, allowing them to ‘fall’ upward and downward at will, navigating between their homes, workplaces, and entertainment hubs. Public transportation consists of levitating shuttles that glide through the air, docking at various suspended stations.\\nIn this world, water must be carefully controlled to avoid floating away, and food production has adapted to vertical farms anchored to the city's central gravity core. The sky, once a distant ceiling, is now the deep abyss below, making nightti

# Speculative Sampling

In [5]:
start = time.time()
tasks = [
    agenerate_text(api_url, model, prompt, speculative_sampling=True)
    for prompt in prompts
]

responses = await asyncio.gather(*tasks)
for i, r in enumerate(responses):
    print(f"Prompt {i+1} response:", r)
print()
print(f"Total time: {time.time()-start:.2f} sec")

Prompt 1 response: {'model_name': 'tensorrt_llm_bls', 'model_version': '1', 'text_output': "Imagine a futuristic city where gravity works in reverse. Instead of buildings rising from the ground, they extend downward into the sky, suspended by powerful energy tethers. Streets are not paved on land but instead consist of floating platforms that drift between these hanging structures, adjusting their positions dynamically to control traffic flow. People move through the city using personal gravity harnesses, allowing them to ‘fall’ upward and downward at will, navigating between their homes, workplaces, and entertainment hubs. Public transportation consists of levitating shuttles that glide through the air, docking at various suspended stations.\\nIn this world, water must be carefully controlled to avoid floating away, and food production has adapted to vertical farms anchored to the city's central gravity core. The sky, once a distant ceiling, is now the deep abyss below, making nightti